In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Import Necessary Libraries

In [ ]:
import cv2
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize , to_categorical
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Dropout ,Flatten , Dense , Activation

#Load the dataset

In [ ]:
no_tumor_scans = os.listdir('/content/drive/MyDrive/Dataset/no') #store all the scans with no tumor in a array
tumor_scans = os.listdir('/content/drive/MyDrive/Dataset/yes')#store all the scans with tumor in a array


#Declare necessary varibles

In [ ]:
dataset = []
label = []

#Data Preprocessing

In [ ]:
for i , image_name in enumerate(no_tumor_scans):
  if(image_name.split('.')[1] == 'jpg'):
    image = cv2.imread('/content/drive/MyDrive/Dataset/no/'+image_name) #reading images
    image = Image.fromarray(image , 'RGB') #convert to RGB format for compatibility with opencv
    image = image.resize((64,64))
    dataset.append(np.array(image))
    label.append(0) #no brain_tumor has label of 0


for i , image_name in enumerate(tumor_scans):
  if(image_name.split('.')[1] == 'jpg'):
    image = cv2.imread('/content/drive/MyDrive/Dataset/yes/'+image_name) #reading images
    image = Image.fromarray(image , 'RGB') #convert to RGB format for compatibility with opencv
    image = image.resize((64,64))
    dataset.append(np.array(image))
    label.append(1) #no brain_tumor has label of 0

In [ ]:
dataset = np.array(dataset)
label = np.array(label)

##Splitting dataset into train and test set

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(dataset , label , test_size = 0.2 , random_state = 0)


##Normalizing the data

In [ ]:
x_train = normalize(x_train)
x_test = normalize(x_test)

##convert y_train and y_test to categorical

In [ ]:
y_train = to_categorical(y_train , num_classes = 2)
y_test = to_categorical(y_test , num_classes = 2)

In [ ]:
y_train.shape

(2400, 2, 2)

#Building the Model

In [56]:
model = Sequential()
model.add(Conv2D(32 , (3,3) , input_shape = (64,64,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32 , (3,3) , kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64 , (3,3) , kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#output layer
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

##Training the Model

In [59]:
model.fit(x_train , y_train , batch_size = 32 , verbose=1 , epochs = 13 , validation_data = (x_test , y_test) , shuffle = False)
model.save('BrainTumor10Epochs.h5')

Epoch 1/13
75/75 [==============================] - 1s 9ms/step - loss: 0.1596 - accuracy: 0.9004 - val_loss: 0.4929 - val_accuracy: 0.8650
Epoch 2/13
75/75 [==============================] - 1s 7ms/step - loss: 0.1515 - accuracy: 0.9083 - val_loss: 0.5180 - val_accuracy: 0.8583
Epoch 3/13
75/75 [==============================] - 1s 9ms/step - loss: 0.1488 - accuracy: 0.9142 - val_loss: 0.5339 - val_accuracy: 0.8717
Epoch 4/13
75/75 [==============================] - 1s 10ms/step - loss: 0.1546 - accuracy: 0.9033 - val_loss: 0.5039 - val_accuracy: 0.8683
Epoch 5/13
75/75 [==============================] - 1s 9ms/step - loss: 0.1507 - accuracy: 0.9092 - val_loss: 0.4855 - val_accuracy: 0.8617
Epoch 6/13
75/75 [==============================] - 1s 10ms/step - loss: 0.1507 - accuracy: 0.9104 - val_loss: 0.5776 - val_accuracy: 0.8617
Epoch 7/13
75/75 [==============================] - 1s 9ms/step - loss: 0.1525 - accuracy: 0.9083 - val_loss: 0.6270 - val_accuracy: 0.8483
Epoch 8/13
75/75 [

#Testing the model

In [48]:
test_image = cv2.imread('/content/drive/MyDrive/pred/pred0.jpg') #no_tumor_scan
img = Image.fromarray(test_image)
img = img.resize((64,64))
img = np.array(img)
input_img = np.expand_dims(img,axis=0)
result1 = model.predict(input_img)
print(result1)

1/1 [==============================] - 0s 19ms/step
[[0.]]
